In [44]:
# Learning torch vision


In [45]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages")))) # load the images
        
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks")))) # load the masks
    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx]) # extract current path to the images
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])# extract current path to the masks
        img = Image.open(img_path).convert("RGB") # convert to the RGB scale
        mask = Image.open(mask_path) # we may not be able to open all of the images, should check size range
        mask = np.array(mask) # convert PIL data into a numpy array
        obj_ids = np.unique(mask) # each value corresponds to one color
        # first id is the background, so remove it
        obj_ids = obj_ids[1:] #I am not sure how this knows the value is in the background
        
        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None] # the masks list contains tiles with and without masks
        
        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax]) # return the index of the tiles with positive values for the masks
        
        #Convert the boxes into a tensor
    
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        
        # make the array of masks into a torch tensor
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        # set the current image ID
        image_id = torch.tensor([idx])
        
        # total area of the boxes where the mask was drawn
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        # suppose all instances are not crowd... i.e. not lots of people
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)
        
print("done")

done


In [46]:
import torchvision

# from torch vision get a recurrent convolutional neural network pretrained on resnet 50
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained input layer with our new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [47]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280


# this code will make multiple aspect ratios for the images

# RPN generate 5 x 3 anchors per spatial location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios


anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),)) #We can increase or decrease the number of aspects if we choose too


In [48]:
# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. This is confusuing and we should read into this. I am not sure why my expected value is [0].

# the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use. # why zero though?
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

# push to the model: transfer learning from FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)



In [49]:
# now we need to fine tune the big model to the small data we have.
# In this case we will run the segmentation task on the images that have groups of people in them
# then return the tuned model

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    return model


In [50]:
# That’s it, this will make model be ready to be trained and evaluated on your custom dataset.

import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)




In [51]:
import utils

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True) # out model is the fasterRCNN
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True)) #out data set will be the people in the fudan 
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn) #Data loader will suffle the data and use 2 cpus, need to look up the collate function
# For Training
images,targets = next(iter(data_loader)) # we have an X and a Y. Images and the people as the masked targets
images = list(image for image in images) # images become a list of images
targets = [{k: v for k, v in t.items()} for t in targets] # out target labels, this makes a list of tensors. using list comprhenesion we 
# iterate over a dict to assign the t'th target to be the key k in assigned to the value v from the t items.
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3137: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


In [52]:
from engine import train_one_epoch, evaluate
import utils
import coco_utils
import coco_eval


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # i think this will work...

    # our dataset has two classes only - background and person
    num_classes = 2 # this will change with the number of options
    # use our dataset and defined transformations
    dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

In [53]:
main()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /home/mskaro1/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3137: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Epoch: [0]  [ 0/60]  eta: 0:01:18  lr: 0.000090  loss: 4.5794 (4.5794)  loss_classifier: 0.6917 (0.6917)  loss_box_reg: 0.2153 (0.2153)  loss_mask: 3.6647 (3.6647)  loss_objectness: 0.0067 (0.0067)  loss_rpn_box_reg: 0.0011 (0.0011)  time: 1.3065  data: 0.8241  max mem: 2132
Epoch: [0]  [10/60]  eta: 0:00:23  lr: 0.000936  loss: 1.3195 (2.1744)  loss_classifier: 0.4117 (0.4276)  loss_box_reg: 0.2524 (0.2785)  loss_mask: 0.7604 (1.4446)  loss_objectness: 0.0190 (0.0196)  loss_rpn_box_reg: 0.0030 (0.0040)  time: 0.4648  data: 0.0811  max mem: 3500
Epoch: [0]  [20/60]  eta: 0:00:16  lr: 0.001783  loss: 0.9601 (1.5056)  loss_classifier: 0.2378 (0.3082)  loss_box_reg: 0.2554 (0.2680)  loss_mask: 0.3639 (0.8990)  loss_objectness: 0.0223 (0.0242)  loss_rpn_box_reg: 0.0037 (0.0062)  time: 0.3793  data: 0.0069  max mem: 3500
Epoch: [0]  [30/60]  eta: 0:00:12  lr: 0.002629  loss: 0.6328 (1.2088)  loss_classifier: 0.0953 (0.2375)  loss_box_reg: 0.2428 (0.2621)  loss_mask: 0.2251 (0.6822)  loss_ob

In [55]:
print(model)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256)
          (relu): ReLU(inplace=True)
          (downsample)